# код


In [1]:
G1 = {(1,1):1, (1,2):2, (1,3):3, (1,4):4, 
      (2,1):2, (2,2):1, (2,3):4, (2,4):3, 
      (3,1):3, (3,2):4, (3,3):1, (3,4):2, 
      (4,1):4, (4,2):3, (4,3):2, (4,4):1 }
G2 = [[1,2,3,4],
      [2,1,4,3],
      [3,4,1,2],
      [4,3,2,1]]

In [2]:
def toMap(G):
    if type(G)==map: return G
    G1 = {}
    for i in range(len(G)):
        for j in range(len(G[i])):
            if type(G[i][j])==int and G[i][j]!=0:
                G1[(i+1,j+1)]=G[i][j]
    return G1

def ijmax(G):
    imax=0; jmax=0
    for k in G:
        imax = max(imax,k[0])
        jmax = max(jmax,k[1])
    return imax, jmax

def toArr(G):
    if type(G)==list: return G
    imax, jmax = ijmax(G)
    G1 = [[0 for j in range(jmax)] for i in range(imax)]
    for k in G:
        i=k[0]-1
        j=k[1]-1
        G1[i][j]=G[k]
    return G1

In [3]:
toArr(toMap(G2))

[[1, 2, 3, 4], [2, 1, 4, 3], [3, 4, 1, 2], [4, 3, 2, 1]]

In [4]:
max(ijmax(G1))

4

In [5]:
def printG(G):
    for line in toArr(G):
        print(*line)


In [6]:
DEBUG = True
VERBOSE = True
def check(G):
    r = max(ijmax(G))
    for a in range(1,r+1):
        for b in range(1,r+1):
            for c in range(1,r+1):
                if (a,b)in G and (b,c)in G:
                    ab=G[(a,b)]; bc = G[(b,c)]
                    if (ab,c)in G and (a,bc) in G:
                        if G[(ab,c)]!=G[(a,bc)]:
                            if VERBOSE: print('abc!=abc',a,b,c)
                            return False
    return True
def fillId(G,N):
    G[(1,1)]=1
    for i in range(2,N+1):
        G[(1,i)]=i
        G[(i,1)]=i
    return G
def fillInv(G,N):
    '''делает так, чтоб в каждой строке/столбце была ровно одна единица'''
    for i in range(1,N+1):
        c=0;p=0
        for j in range(1,N+1):
            if (i,j) not in G:
                c+=1
                p=j
            elif G[(i,j)]==1:
                c+=1
                p=0
                break
        if c==0:
            if DEBUG: print(i,'* x != 1 for Any x')
            return False
        if p!=0 and c==1:
            G[(i,p)]=1
    for i in range(1,N+1):
        c=0;p=0
        for j in range(1,N+1):
            if (j,i) not in G:
                c+=1
                p=j
            elif G[(j,i)]==1:
                c+=1
                p=0
                break
        if c==0:
            if DEBUG: print(i,'* x != 1 for Any x')
            return False
        if p!=0 and c==1:
            G[(p,i)]=1
    return True
def fill(G,N):
    #fillInv(G,N)
    changed = True
    while changed:
        changed = False
        for i in range(1, N+1):
            for j in range(1,N+1):
                if (i,j) not in G:
                    candidates = set()
                    for k in range(1,N+1):
                        G[(i,j)]=k
                        if VERBOSE: print('assume',i,j,k)
                        #fillInv(G,N)
                        if check(G):
                            candidates.add(k)
                    del G[(i,j)]
                    if len(candidates)==0:
                        if DEBUG: print(i,'*',j,'has no candidates')
                        return False
                    elif len(candidates)==1:
                        if VERBOSE: print('setup ',i,j,next(iter(candidates)))
                        G[(i,j)]=next(iter(candidates))
                        changed = True
    return True

In [7]:
G4 = fillId({},4)
for line in toArr(G4):
    print(*line)


1 2 3 4
2 0 0 0
3 0 0 0
4 0 0 0


In [8]:
import itertools
import copy

def genId(N):
    '''генерирует список групп с расставленными по всякому единицами'''
    Gs = []
    global DEBUG
    DEBUG = False
    for p in itertools.permutations(range(2,N+1)):
        G = fillId({},N)
        for i in range(2,N+1):
            G[(i,p[i-2])]=1
        if fill(G,N):
#            c = 0
#            if (2,2) in G:
#                for k in range(1,N+1):
#                    G1 = copy.copy(G)
#                    G1[(2,3)] = k
#                    if fill(G1,N): c+=1
#            else:
#                for k in range(1,N+1):
#                    G1 = copy.copy(G)
#                    G1[(2,2)] = k
#                    if fill(G1,N): c+=1
#            if c>0:
                Gs.append(G)
    DEBUG = True
    return Gs

In [9]:
def genFill(G,N,ii=1,sets=None):
    '''начиная с ii-й строки находит пустоую клетку и начинает туда подставлять значения, и рекурсивно запускать себя
    а когда не находит пустую клетку - выводит всю группу'''
    global DEBUG
    DEBUG = False
    if sets==None: sets = []
    isClean = True # в нем нет дыр
    
    i1=0; j1=0
    for i in range(ii,N+1):
        for j in range(1,N+1):
            if (i,j) not in G:
                #print(sets,i,j)
                isClean = False
                i1=i; j1=j
                break
        if not isClean: break
    #print(sets,ii,i1,j1)
    if isClean:
        print(sets)
        for line in toArr(G):
            print(*line)
        print()
        return 1
    else:
        count = 0
        #print(i1,j1)
        for k in range(1,N+1):
            G1 = copy.copy(G)
            G1[(i1,j1)]=k
            if fill(G1,N):
                count += genFill(G1,N,i1,[*sets,(i1,j1,k)])
                DEBUG = False
        DEBUG = True
        return count

# left inverse, left 1

In [10]:
G4 = {}
G4[(1,1)]=1

G4[(1,2)]=2
G4[(1,3)]=3
G4[(1,4)]=4
G4[(1,5)]=5
G4[(1,6)]=6

G4[(2,3)]=1
G4[(3,4)]=1
G4[(2,2)]=5

VERBOSE = False
fill(G4,6)
printG(G4)
print()

#G4[(2,2)]=6
#G4[(2,4)]=3
#G4[(4,4)]=2
#VERBOSE = False
#fill(G4,6)
#printG(G4)

1 2 3 4 5 6
4 5 1 3 6 2
3 6 4 1 2 5
4 5 1 3 6 2
3 6 4 1 2 5
1 2 3 4 5 6



In [11]:
G5 = copy.copy(G4)
#G5[(1,3)]=0
fill(G5,8)
printG(G5)

1 2 3 4 5 6
4 5 1 3 6 2
3 6 4 1 2 5
4 5 1 3 6 2
3 6 4 1 2 5
1 2 3 4 5 6


# left-right

In [12]:
G4 = {}
G4[(1,1)]=1

G4[(1,2)]=2
G4[(1,3)]=3
G4[(1,4)]=4
G4[(1,5)]=5
G4[(1,6)]=6
G4[(1,7)]=7
G4[(1,8)]=8
G4[(1,9)]=9
G4[(1,10)]=10

G4[(2,3)]=1
G4[(3,4)]=1
#G4[(4,5)]=1
#G4[(5,6)]=1
#G4[(6,7)]=1
printG(G4)
print()

VERBOSE = False
fill(G4,10)
printG(G4)
print()

G4[(3,2)]=5
#G4[(4,4)]=2
VERBOSE = True
fill(G4,10)
printG(G4)

1 2 3 4 5 6 7 8 9 10
0 0 1 0 0 0 0 0 0 0
0 0 0 1 0 0 0 0 0 0

1 2 3 4 5 6 7 8 9 10
4 0 1 0 0 0 0 0 0 0
3 0 0 1 0 0 0 0 0 0
4 0 1 0 0 0 0 0 0 0

assume 2 2 1
abc!=abc 2 2 2
assume 2 2 2
abc!=abc 2 2 3
assume 2 2 3
abc!=abc 2 2 2
assume 2 2 4
abc!=abc 2 2 3
assume 2 2 5
assume 2 2 6
assume 2 2 7
assume 2 2 8
assume 2 2 9
assume 2 2 10
assume 2 4 1
abc!=abc 2 4 3
assume 2 4 2
abc!=abc 2 4 1
assume 2 4 3
assume 2 4 4
abc!=abc 2 4 3
assume 2 4 5
assume 2 4 6
assume 2 4 7
assume 2 4 8
assume 2 4 9
assume 2 4 10
assume 2 5 1
abc!=abc 2 3 2
assume 2 5 2
assume 2 5 3
abc!=abc 2 3 2
assume 2 5 4
abc!=abc 2 3 2
assume 2 5 5
abc!=abc 2 3 2
assume 2 5 6
abc!=abc 2 3 2
assume 2 5 7
abc!=abc 2 3 2
assume 2 5 8
abc!=abc 2 3 2
assume 2 5 9
abc!=abc 2 3 2
assume 2 5 10
abc!=abc 2 3 2
setup  2 5 2
assume 2 6 1
assume 2 6 2
assume 2 6 3
assume 2 6 4
assume 2 6 5
assume 2 6 6
assume 2 6 7
assume 2 6 8
assume 2 6 9
assume 2 6 10
assume 2 7 1
assume 2 7 2
assume 2 7 3
assume 2 7 4
assume 2 7 5
assume 2 7 6
a

# 2,3,4

In [13]:
r = 2
G4 = fillId({},r)
fillInv(G4,r)
fill(G4,r)
for line in toArr(G4):
    print(*line)

1 2
2 1


In [14]:
r = 3
G4 = fillId({},r)
G4[(2,2)]=3
fillInv(G4,r)
fill(G4,r)
fillInv(G4,r)
for line in toArr(G4):
    print(*line)

assume 3 3 1
abc!=abc 2 2 3
assume 3 3 2
assume 3 3 3
abc!=abc 2 2 3
setup  3 3 2
1 2 3
2 3 1
3 1 2


In [15]:
r = 4
G4 = fillId({},r)
G4[(2,2)]=1
G4[(2,3)]=4
G4[(3,3)]=1

fillInv(G4,r)
fill(G4,r)
fillInv(G4,r)
for line in toArr(G4):
    print(*line)

assume 2 4 1
abc!=abc 2 2 3
assume 2 4 2
abc!=abc 2 2 3
assume 2 4 3
assume 2 4 4
abc!=abc 2 2 3
setup  2 4 3
assume 3 2 1
abc!=abc 3 2 2
assume 3 2 2
abc!=abc 3 2 2
assume 3 2 3
abc!=abc 3 3 2
assume 3 2 4
setup  3 2 4
assume 3 4 1
abc!=abc 3 3 2
assume 3 4 2
assume 3 4 3
abc!=abc 3 3 2
assume 3 4 4
abc!=abc 3 3 2
setup  3 4 2
assume 4 2 1
abc!=abc 2 3 2
assume 4 2 2
abc!=abc 2 3 2
assume 4 2 3
assume 4 2 4
abc!=abc 2 3 2
setup  4 2 3
assume 4 3 1
abc!=abc 2 3 3
assume 4 3 2
assume 4 3 3
abc!=abc 2 3 3
assume 4 3 4
abc!=abc 2 3 3
setup  4 3 2
assume 4 4 1
assume 4 4 2
abc!=abc 2 3 4
assume 4 4 3
abc!=abc 2 3 4
assume 4 4 4
abc!=abc 2 3 4
setup  4 4 1
1 2 3 4
2 1 4 3
3 4 1 2
4 3 2 1


In [16]:
r = 4
G4 = fillId({},r)
G4[(2,2)]=1
G4[(2,3)]=4
G4[(3,3)]=2

fillInv(G4,r)
fill(G4,r)
fillInv(G4,r)
for line in toArr(G4):
    print(*line)

assume 2 4 1
abc!=abc 2 2 3
assume 2 4 2
abc!=abc 2 2 3
assume 2 4 3
assume 2 4 4
abc!=abc 2 2 3
setup  2 4 3
assume 3 2 1
abc!=abc 3 2 2
assume 3 2 2
abc!=abc 3 2 2
assume 3 2 3
abc!=abc 3 3 2
assume 3 2 4
setup  3 2 4
assume 3 4 1
assume 3 4 2
abc!=abc 3 2 3
assume 3 4 3
abc!=abc 3 2 3
assume 3 4 4
abc!=abc 3 2 3
setup  3 4 1
assume 4 2 1
abc!=abc 2 3 2
assume 4 2 2
abc!=abc 2 3 2
assume 4 2 3
assume 4 2 4
abc!=abc 2 3 2
setup  4 2 3
assume 4 4 1
abc!=abc 2 3 4
assume 4 4 2
assume 4 4 3
abc!=abc 2 3 4
assume 4 4 4
abc!=abc 2 3 4
setup  4 4 2
1 2 3 4
2 1 4 3
3 4 2 1
4 3 1 2


In [17]:
r = 4
G4 = fillId({},r)
G4[(2,2)]=3
G4[(2,3)]=4

fillInv(G4,r)
fill(G4,r)
fillInv(G4,r)
for line in toArr(G4):
    print(*line)

assume 3 2 1
abc!=abc 2 2 2
assume 3 2 2
abc!=abc 2 2 2
assume 3 2 3
abc!=abc 2 2 2
assume 3 2 4
setup  3 2 4
assume 3 3 1
assume 3 3 2
abc!=abc 2 2 3
assume 3 3 3
abc!=abc 2 2 3
assume 3 3 4
abc!=abc 2 2 3
setup  3 3 1
assume 3 4 1
abc!=abc 2 2 4
assume 3 4 2
assume 3 4 3
abc!=abc 2 2 4
assume 3 4 4
abc!=abc 2 2 4
setup  3 4 2
assume 4 2 1
assume 4 2 2
abc!=abc 2 3 2
assume 4 2 3
abc!=abc 2 3 2
assume 4 2 4
abc!=abc 2 3 2
setup  4 2 1
assume 4 3 1
abc!=abc 2 3 3
assume 4 3 2
assume 4 3 3
abc!=abc 2 3 3
assume 4 3 4
abc!=abc 2 3 3
setup  4 3 2
assume 4 4 1
abc!=abc 2 3 4
assume 4 4 2
abc!=abc 2 3 4
assume 4 4 3
assume 4 4 4
abc!=abc 2 3 4
setup  4 4 3
1 2 3 4
2 3 4 1
3 4 1 2
4 1 2 3


In [18]:
r = 4
G4 = fillId({},r)
G4[(2,2)]=4
G4[(2,3)]=1

fillInv(G4,r)
fill(G4,r)
fillInv(G4,r)
for line in toArr(G4):
    print(*line)

assume 2 4 1
assume 2 4 2
assume 2 4 3
assume 2 4 4
assume 3 2 1
assume 3 2 2
abc!=abc 2 3 2
assume 3 2 3
abc!=abc 2 3 2
assume 3 2 4
assume 3 3 1
abc!=abc 2 3 3
assume 3 3 2
abc!=abc 2 3 3
assume 3 3 3
abc!=abc 2 3 3
assume 3 3 4
setup  3 3 4
assume 3 4 1
abc!=abc 2 3 4
assume 3 4 2
assume 3 4 3
abc!=abc 2 3 4
assume 3 4 4
assume 4 2 1
abc!=abc 4 2 3
assume 4 2 2
abc!=abc 4 2 3
assume 4 2 3
assume 4 2 4
assume 4 3 1
abc!=abc 2 2 3
assume 4 3 2
assume 4 3 3
abc!=abc 2 2 3
assume 4 3 4
abc!=abc 2 2 3
setup  4 3 2
assume 4 4 1
assume 4 4 2
abc!=abc 4 3 3
assume 4 4 3
abc!=abc 4 3 3
assume 4 4 4
abc!=abc 4 3 3
setup  4 4 1
assume 2 4 1
abc!=abc 2 2 4
assume 2 4 2
abc!=abc 2 2 4
assume 2 4 3
assume 2 4 4
abc!=abc 2 2 4
setup  2 4 3
assume 3 2 1
assume 3 2 2
abc!=abc 2 3 2
assume 3 2 3
abc!=abc 2 3 2
assume 3 2 4
abc!=abc 2 3 2
setup  3 2 1
assume 3 4 1
abc!=abc 2 3 4
assume 3 4 2
assume 3 4 3
abc!=abc 2 3 4
assume 3 4 4
abc!=abc 2 3 4
setup  3 4 2
assume 4 2 1
abc!=abc 2 2 2
assume 4 2 2
a